In [4]:
pip install spotipy pandas python-dotenv

Note: you may need to restart the kernel to use updated packages.


In [7]:
import csv
import os
from dotenv import load_dotenv
from spotipy.oauth2 import SpotifyClientCredentials
import spotipy

# .envファイルから環境変数をロード
load_dotenv()

# Spotify APIの認証情報を環境変数から取得
CLIENT_ID = os.getenv('SPOTIFY_CLIENT_ID')
CLIENT_SECRET = os.getenv('SPOTIFY_CLIENT_SECRET')

if not CLIENT_ID or not CLIENT_SECRET:
    raise ValueError("Spotify APIの認証情報が見つかりません。SPOTIFY_CLIENT_IDとSPOTIFY_CLIENT_SECRETを.envに設定してください。")

# Spotify APIクライアントの認証
auth_manager = SpotifyClientCredentials(client_id=CLIENT_ID, client_secret=CLIENT_SECRET)
spotify = spotipy.Spotify(auth_manager=auth_manager)

def fetch_spotify_artist_id(artist_name):
    """Spotify APIを使用してアーティストIDを取得する"""
    try:
        search_results = spotify.search(q=f"artist:{artist_name}", type="artist", limit=1)
        artists = search_results.get('artists', {}).get('items', [])
        if artists:
            return artists[0]['id']  # 最初の結果のIDを返す
        else:
            print(f"アーティストが見つかりません: {artist_name}")
            return None
    except Exception as e:
        print(f"Spotify APIエラー: {e}")
        return None

def update_artist_ids(input_csv, output_csv):
    """rappers.csvを更新してSpotify内部IDを設定"""
    updated_rows = []

    # 入力CSVを読み込む
    with open(input_csv, 'r', encoding='utf-8') as infile:
        reader = csv.DictReader(infile)
        for row in reader:
            artist_name = row['name']
            print(f"Fetching Spotify ID for artist: {artist_name}")
            spotify_id = fetch_spotify_artist_id(artist_name)
            if spotify_id:
                row['id'] = spotify_id  # IDをSpotifyの内部IDに置き換え
            updated_rows.append(row)

    # 更新されたデータを新しいCSVに書き込む
    with open(output_csv, 'w', newline='', encoding='utf-8') as outfile:
        fieldnames = ['id', 'name']
        writer = csv.DictWriter(outfile, fieldnames=fieldnames)
        writer.writeheader()
        writer.writerows(updated_rows)

    print(f"Spotify IDに更新されたCSVが保存されました: {output_csv}")

if __name__ == "__main__":
    # 入力CSVファイルと出力CSVファイルのパスを指定
    input_csv_path = './rappers.csv'
    output_csv_path = './rappers_with_spotify_ids.csv'

    # アーティストIDを更新
    update_artist_ids(input_csv_path, output_csv_path)


Fetching Spotify ID for artist: Kanye West
Fetching Spotify ID for artist: Jay-Z
Fetching Spotify ID for artist: Eminem
Fetching Spotify ID for artist: Drake
Fetching Spotify ID for artist: Kendrick Lamar
Fetching Spotify ID for artist: Travis Scott
Fetching Spotify ID for artist: Nicki Minaj
Fetching Spotify ID for artist: Cardi B
Fetching Spotify ID for artist: Lil Wayne
Fetching Spotify ID for artist: Snoop Dogg
Fetching Spotify ID for artist: 50 Cent
Fetching Spotify ID for artist: Future
Fetching Spotify ID for artist: Post Malone
Fetching Spotify ID for artist: A$AP Rocky
Fetching Spotify ID for artist: J. Cole
Fetching Spotify ID for artist: Childish Gambino
Fetching Spotify ID for artist: Tyler, The Creator
Fetching Spotify ID for artist: Ice Cube
Fetching Spotify ID for artist: Megan Thee Stallion
Fetching Spotify ID for artist: Doja Cat
Fetching Spotify ID for artist: Nas
Fetching Spotify ID for artist: Lil Nas X
Fetching Spotify ID for artist: 21 Savage
Fetching Spotify ID f

In [11]:
import csv
import os
from dotenv import load_dotenv
from spotipy.oauth2 import SpotifyClientCredentials
import spotipy

# .envファイルから環境変数をロード
load_dotenv()

# Spotify APIの認証情報を環境変数から取得
CLIENT_ID = os.getenv('SPOTIFY_CLIENT_ID')
CLIENT_SECRET = os.getenv('SPOTIFY_CLIENT_SECRET')

if not CLIENT_ID or not CLIENT_SECRET:
    raise ValueError("Spotify APIの認証情報が見つかりません。SPOTIFY_CLIENT_IDとSPOTIFY_CLIENT_SECRETを.envに設定してください。")

# Spotify APIクライアントの認証
auth_manager = SpotifyClientCredentials(client_id=CLIENT_ID, client_secret=CLIENT_SECRET)
spotify = spotipy.Spotify(auth_manager=auth_manager)

def get_tracks_with_features(artist_name, artist_id):
    """アーティスト名からフィーチャリングされている楽曲を取得"""
    search_results = spotify.search(q=f'artist:{artist_name}', type='track', limit=50)
    tracks = []

    for track in search_results['tracks']['items']:
        main_artist_id = artist_id  # メインアーティストのID
        # フィーチャリングアーティストを取得
        featured_artist_ids = [artist['id'] for artist in track['artists'][1:]]
        if featured_artist_ids:
            track_info = {
                'id': track['id'],
                'release_year': track['album']['release_date'].split('-')[0],
                'artists': [main_artist_id] + featured_artist_ids,
                'track_name': track['name']
            }
            tracks.append(track_info)
    return tracks

def main():
    # CSVファイルからラッパー名とIDを読み込み
    with open('./rappers_with_spotify_ids.csv', 'r', encoding='utf-8') as csvfile:
        reader = csv.DictReader(csvfile)
        rappers = [{'id': row['id'], 'name': row['name']} for row in reader]

    all_tracks = []

    # 各ラッパーの楽曲を取得
    for rapper in rappers:
        print(f"Fetching tracks for {rapper['name']}...")
        tracks = get_tracks_with_features(rapper['name'], rapper['id'])
        all_tracks.extend(tracks)

    # 曲名で重複を削除
    unique_tracks = {track['track_name']: track for track in all_tracks}.values()

    # 結果をCSVに保存
    with open('featured_tracks.csv', 'w', newline='', encoding='utf-8') as csvfile:
        fieldnames = ['id', 'release_year', 'artists', 'track_name']
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
        writer.writeheader()
        for track in unique_tracks:
            writer.writerow({
                'id': track['id'],
                'release_year': track['release_year'],
                'artists': track['artists'],  # IDの配列をそのまま保存
                'track_name': track['track_name']
            })

    print("フィーチャリング楽曲のCSVファイルが 'featured_tracks.csv' に保存されました。")

if __name__ == "__main__":
    main()


Fetching tracks for Kanye West...
Fetching tracks for Jay-Z...
Fetching tracks for Eminem...
Fetching tracks for Drake...
Fetching tracks for Kendrick Lamar...
Fetching tracks for Travis Scott...
Fetching tracks for Nicki Minaj...
Fetching tracks for Cardi B...
Fetching tracks for Lil Wayne...
Fetching tracks for Snoop Dogg...
Fetching tracks for 50 Cent...
Fetching tracks for Future...
Fetching tracks for Post Malone...
Fetching tracks for A$AP Rocky...
Fetching tracks for J. Cole...
Fetching tracks for Childish Gambino...
Fetching tracks for Tyler, The Creator...
Fetching tracks for Ice Cube...
Fetching tracks for Megan Thee Stallion...
Fetching tracks for Doja Cat...
Fetching tracks for Nas...
Fetching tracks for Lil Nas X...
Fetching tracks for 21 Savage...
Fetching tracks for Juice WRLD...
Fetching tracks for Lil Baby...
Fetching tracks for Jack Harlow...
Fetching tracks for Playboi Carti...
Fetching tracks for Young Thug...
Fetching tracks for Roddy Ricch...
Fetching tracks for L